In [2]:
import numpy as np
import pandas as pd

In [3]:
# read the csv file
house_members = pd.read_csv('../data/H118_members.csv')
senate_members = pd.read_csv('../data/S118_members.csv')

house_votes = pd.read_csv('../data/H118_rollcalls_CLEANSED.csv')
senate_votes = pd.read_csv('../data/S118_rollcalls_CLEANSED.csv')

In [37]:
def predict_vote(member_df, vote_df, member_id, bill_id, k=5):
    member = member_df[member_df['icpsr'] == member_id]
    bill = vote_df[vote_df['rollnumber'] == bill_id]
    
    member_nominal_points = member[['nominate_dim1', 'nominate_dim2']].values[0]
    bill_nominal_points = bill[['nominate_mid_1', 'nominate_mid_2']].values[0]
    print(member_nominal_points)
    
    

In [54]:
bill = house_votes[house_votes['rollnumber'] == 118]
bill_nominal_points = bill[['nominate_mid_1', 'nominate_mid_2']].values[0]
print('bill points', bill_nominal_points)

# predict_vote(house_members, house_votes, member_id=14854, bill_id=118, k=5)
# predict_vote(house_members, house_votes, member_id=14873, bill_id=118, k=5)


house_mat = pd.read_csv('house_cf_118.csv', index_col=0)
# get members who voted 'Yea' on bill 118 (where 118 is the column index)
yea_voters = house_mat[house_mat['118'] == 1]
print('Yea voters on bill 118:', yea_voters.index.tolist())

distances = []
for member_id in yea_voters.index.tolist():
    #predict_vote(house_members, house_votes, member_id=member_id, bill_id=118, k=5)
    
    # get distance from bill nominal points
    member = house_members[house_members['icpsr'] == member_id]
    member_nominal_points = member[['nominate_dim1', 'nominate_dim2']].values[0]
    distance = np.linalg.norm(member_nominal_points - bill_nominal_points)
    #print(f'Member ID: {member_id}, Distance from bill points: {distance}')
    
    distances.append((member_id, distance))
    
average_distance = np.mean([d[1] for d in distances])
print('Average distance of Yea voters from bill nominal points:', average_distance)

nay_voters = house_mat[house_mat['118'] == -1]
print('Nay voters on bill 118:', nay_voters.index.tolist())

distances = []
for member_id in nay_voters.index.tolist():
    #predict_vote(house_members, house_votes, member_id=member_id, bill_id=118, k=5)
    
    # get distance from bill nominal points
    member = house_members[house_members['icpsr'] == member_id]
    member_nominal_points = member[['nominate_dim1', 'nominate_dim2']].values[0]
    distance = np.linalg.norm(member_nominal_points - bill_nominal_points)
    #print(f'Member ID: {member_id}, Distance from bill points: {distance}')
    
    distances.append((member_id, distance))
    
average_distance = np.mean([d[1] for d in distances])
print('Average distance of Yea voters from bill nominal points:', average_distance)


bill points [-0.261  0.26 ]
Yea voters on bill 118: [14854, 14863, 15029, 20107, 20124, 20138, 20301, 20316, 20342, 20344, 20355, 20356, 20501, 20521, 20522, 20530, 20533, 20535, 20703, 20704, 20709, 20725, 20729, 20738, 20755, 20756, 20758, 20759, 20903, 20909, 20916, 20926, 20946, 21103, 21105, 21106, 21108, 21116, 21123, 21132, 21139, 21142, 21162, 21167, 21178, 21179, 21191, 21196, 21302, 21307, 21333, 21337, 21346, 21351, 21353, 21356, 21360, 21364, 21367, 21368, 21373, 21500, 21503, 21510, 21513, 21515, 21516, 21519, 21523, 21526, 21531, 21532, 21541, 21544, 21551, 21556, 21557, 21559, 21561, 21562, 21563, 21564, 21565, 21700, 21701, 21702, 21704, 21705, 21711, 21714, 21717, 21718, 21719, 21720, 21722, 21723, 21724, 21727, 21731, 21735, 21740, 21744, 21745, 21749, 21750, 21753, 21755, 21757, 21758, 21759, 21760, 21901, 21903, 21905, 21908, 21910, 21911, 21920, 21923, 21925, 21926, 21927, 21930, 21935, 21936, 21938, 21945, 21946, 21951, 21952, 21953, 21956, 21959, 21961, 21962, 21

In [55]:
import numpy as np
import pandas as pd

def avg_vote_distances(house_members, house_votes, vote_matrix, bill_id):
    bill = house_votes[house_votes['rollnumber'] == bill_id]
    bill_midpoint = bill[['nominate_mid_1', 'nominate_mid_2']].values[0]

    def compute_avg_for(vote_value):
        voters = vote_matrix[vote_matrix[str(bill_id)] == vote_value].index.tolist()
        distances = []

        for member_id in voters:
            member = house_members[house_members['icpsr'] == member_id]
            
            if member.empty:
                continue 
            
            m_point = member[['nominate_dim1', 'nominate_dim2']].values[0]
            dist = np.linalg.norm(m_point - bill_midpoint)
            distances.append(dist)

        return np.mean(distances) if distances else np.nan

    return {
        "yea_avg": compute_avg_for(1),
        "nay_avg": compute_avg_for(-1)
    }


In [64]:
for col in house_mat.columns:
    if col.isdigit():
        bill_id = int(col)
        averages = avg_vote_distances(house_members, house_votes, house_mat, bill_id)
        print(f'Bill ID: {bill_id}, Yea Avg Distance: {averages["yea_avg"]}, Nay Avg Distance: {averages["nay_avg"]}')

Bill ID: 103, Yea Avg Distance: 0.9572838330420188, Nay Avg Distance: 0.9604758871705535
Bill ID: 118, Yea Avg Distance: 0.7560125470393412, Nay Avg Distance: 0.4439611674312336
Bill ID: 123, Yea Avg Distance: 0.6312302666457303, Nay Avg Distance: 0.5183333281361097
Bill ID: 141, Yea Avg Distance: 0.5902453512585726, Nay Avg Distance: 0.6499463110973592
Bill ID: 187, Yea Avg Distance: 0.7686719891545535, Nay Avg Distance: 0.5047372245342955
Bill ID: 201, Yea Avg Distance: 0.7373451978600207, Nay Avg Distance: 0.9365125923512626
Bill ID: 231, Yea Avg Distance: 0.5550849489081278, Nay Avg Distance: 0.5342969146867933
Bill ID: 233, Yea Avg Distance: 0.6769345922849532, Nay Avg Distance: 0.5297532642092861
Bill ID: 242, Yea Avg Distance: 0.6213763768551758, Nay Avg Distance: 0.5851399952614414
Bill ID: 251, Yea Avg Distance: 0.53330395352717, Nay Avg Distance: 0.5809262418299643
Bill ID: 363, Yea Avg Distance: 1.097247742334106, Nay Avg Distance: 1.3542860252477846
Bill ID: 379, Yea Avg Di

In [ ]:
def find_optimal_threshold(house_members, house_votes, vote_matrix):
    all_distances_yea = []
    all_distances_nay = []
    
    for bill_id in vote_matrix.columns:
        bill_id_int = int(bill_id)
        bill = house_votes[house_votes['rollnumber'] == bill_id_int]
        if bill.empty:
            continue
            
        bill_midpoint = bill[['nominate_mid_1', 'nominate_mid_2']].values[0]
        
        # fidtance for yay votes
        yea_voters = vote_matrix[vote_matrix[bill_id] == 1].index.tolist()
        for member_id in yea_voters:
            member = house_members[house_members['icpsr'] == member_id]
            if not member.empty:
                m_point = member[['nominate_dim1', 'nominate_dim2']].values[0]
                dist = np.linalg.norm(m_point - bill_midpoint)
                all_distances_yea.append(dist)
        
        # find distance for nay votes
        nay_voters = vote_matrix[vote_matrix[bill_id] == -1].index.tolist()
        for member_id in nay_voters:
            member = house_members[house_members['icpsr'] == member_id]
            if not member.empty:
                m_point = member[['nominate_dim1', 'nominate_dim2']].values[0]
                dist = np.linalg.norm(m_point - bill_midpoint)
                all_distances_nay.append(dist)
    
    yea_75th = np.percentile(all_distances_yea, 75)
    nay_25th = np.percentile(all_distances_nay, 25)
    optimal_threshold = (yea_75th + nay_25th) / 2
    
    return optimal_threshold

In [70]:
optimal_threshold = find_optimal_threshold(house_members, house_votes, house_mat)

In [67]:
senate_mat = pd.read_csv('senate_cf_118.csv', index_col=0)
find_optimal_threshold(senate_members, senate_votes, senate_mat)

0.7786803442593813

In [ ]:
def predict_votes_for_new_bill(house_members, new_bill_midpoint, 
                               optimal_threshold, member_ids=None):

    predictions = {}
    
    if member_ids is None:
        member_ids = house_members['icpsr'].tolist()
    
    member_ids = [int(mid) for mid in member_ids]    

    for member_id in member_ids:
        member = house_members[house_members['icpsr'] == member_id]
        if member.empty:
            continue
                
        member_point = member[['nominate_dim1', 'nominate_dim2']].values[0]
        distance = np.linalg.norm(member_point - new_bill_midpoint)

        vote = 1 if distance < optimal_threshold else -1
        confidence = abs(distance - optimal_threshold) / optimal_threshold

        predictions[member_id] = {
            'vote': vote,
            'confidence': confidence,
            'distance': distance
        }
    
    return predictions


In [120]:
bill_nominal_points = house_votes[house_votes['rollnumber'] == 118][['nominate_mid_1', 'nominate_mid_2']].values[0]
bill_spread = house_votes[house_votes['rollnumber'] == 118][['nominate_spread_1', 'nominate_spread_2']].values[0]
predictions = predict_votes_for_new_bill(house_members, bill_nominal_points, 
                                     optimal_threshold)

print(predictions)

{20301: {'vote': 1, 'confidence': 0.10350680686284197, 'distance': 0.6499861536986768}, 21102: {'vote': 1, 'confidence': 0.7327100951444899, 'distance': 0.19379370474811614}, 21500: {'vote': -1, 'confidence': 0.3052257347316049, 'distance': 0.9463302806103164}, 22108: {'vote': -1, 'confidence': 0.11894642539997141, 'distance': 0.8112718410002902}, 22140: {'vote': -1, 'confidence': 0.43128109074570353, 'distance': 1.0377244335564235}, 22366: {'vote': -1, 'confidence': 0.20385428182561752, 'distance': 0.8728327445736669}, 29701: {'vote': 1, 'confidence': 0.03725112132876504, 'distance': 0.6980236385682078}, 22168: {'vote': 1, 'confidence': 0.764907711697222, 'distance': 0.1704494059831245}, 21995: {'vote': 1, 'confidence': 0.18480626534117012, 'distance': 0.5910414537069291}, 20305: {'vote': 1, 'confidence': 0.14070973977108567, 'distance': 0.6230128409591571}, 21103: {'vote': -1, 'confidence': 0.7096627343151198, 'distance': 1.2395600025815612}, 21105: {'vote': -1, 'confidence': 0.30336

In [139]:
strong_predictions = {
    member_id: pred for member_id, pred in predictions.items()
    if pred['confidence'] > 0.7  # Only high-confidence predictions
}

# Display results
for member_id, pred in strong_predictions.items():
    vote_str = "YEA" if pred['vote'] == 1 else "NAY"
    print(f"Member {member_id}: {vote_str} (confidence: {pred['confidence']:.2%})")


Member 21102: YEA (confidence: 73.27%)
Member 22168: YEA (confidence: 76.49%)
Member 21103: NAY (confidence: 70.97%)
Member 21502: YEA (confidence: 79.11%)
Member 21705: NAY (confidence: 88.18%)
Member 21968: YEA (confidence: 97.48%)
Member 22311: NAY (confidence: 88.61%)
Member 20501: YEA (confidence: 72.70%)
Member 21308: YEA (confidence: 70.16%)
Member 21311: YEA (confidence: 70.48%)
Member 21506: YEA (confidence: 81.43%)
Member 21709: YEA (confidence: 76.18%)
Member 21711: YEA (confidence: 87.48%)
Member 21740: YEA (confidence: 89.01%)
Member 21930: YEA (confidence: 73.54%)
Member 21939: YEA (confidence: 84.00%)
Member 21912: YEA (confidence: 84.22%)
Member 22103: NAY (confidence: 72.35%)
Member 22359: YEA (confidence: 80.38%)
Member 21719: NAY (confidence: 71.42%)
Member 21746: YEA (confidence: 85.15%)
Member 22353: YEA (confidence: 89.39%)
Member 20321: YEA (confidence: 85.81%)
Member 21944: YEA (confidence: 84.08%)
Member 22121: NAY (confidence: 88.37%)
Member 29339: YEA (confid

In [140]:
len(strong_predictions)

# sort strong prediction by member_id
strong_predictions = dict(sorted(strong_predictions.items()))

In [141]:
# filter only nay votes
strong_predictions = {k: v for k, v in strong_predictions.items() if v['vote'] == -1}
strong_predictions

{21103: {'vote': -1,
  'confidence': 0.7096627343151198,
  'distance': 1.2395600025815612},
 21705: {'vote': -1,
  'confidence': 0.8817869191244425,
  'distance': 1.3643555255137862},
 21719: {'vote': -1,
  'confidence': 0.7142275828219655,
  'distance': 1.2428696633195293},
 21753: {'vote': -1,
  'confidence': 0.7691005014878628,
  'distance': 1.282654279219463},
 21961: {'vote': -1,
  'confidence': 0.8837344531226334,
  'distance': 1.3657675497682613},
 22103: {'vote': -1,
  'confidence': 0.723514724226861,
  'distance': 1.2496031369999037},
 22120: {'vote': -1,
  'confidence': 0.8837344531226334,
  'distance': 1.3657675497682613},
 22121: {'vote': -1,
  'confidence': 0.8837344531226334,
  'distance': 1.3657675497682613},
 22148: {'vote': -1,
  'confidence': 0.8865594699680397,
  'distance': 1.3678157770694124},
 22303: {'vote': -1,
  'confidence': 0.8374406450060516,
  'distance': 1.3322030625996923},
 22305: {'vote': -1,
  'confidence': 0.7472660240275306,
  'distance': 1.266823586